### MySQL 접속용 샘플 프로그램

- 접속 정보가 들어 있는 파일 읽기

In [3]:
import json
with open('./mysql.json') as f:
    config_str =f.read()
config = json.loads(config_str)
config

{'host': 'localhost',
 'user': 'ysuser',
 'password': 'yspass',
 'database': 'world',
 'port': 3306}

- MySQL에 접속

In [4]:
import pymysql
conn = pymysql.connect(
    host=config['host'],user=config['user'],passwd=config['password'],
    database=config['database'],port=config['port']
)

##### 3. MYSQL Access

In [5]:
# city table에서 10개 데이터 읽기
cur = conn.cursor()
sql = "SELECT * FROM city LIMIT 10;"
cur.execute(sql)
rows =cur.fetchall()
cur.close()

for row in rows:
    print(row)

(1, 'Kabul', 'AFG', 'Kabol', 1780000)
(2, 'Qandahar', 'AFG', 'Qandahar', 237500)
(3, 'Herat', 'AFG', 'Herat', 186800)
(4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800)
(5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200)
(6, 'Rotterdam', 'NLD', 'Zuid-Holland', 593321)
(7, 'Haag', 'NLD', 'Zuid-Holland', 440900)
(8, 'Utrecht', 'NLD', 'Utrecht', 234323)
(9, 'Eindhoven', 'NLD', 'Noord-Brabant', 201843)
(10, 'Tilburg', 'NLD', 'Noord-Brabant', 193238)


In [8]:
import pandas as pd
df = pd.DataFrame(rows,columns=['id','name','country_code','district','population'])
df.set_index('id',inplace=True)
df

,name,country_code,district,population
id,,,,
1,Kabul,AFG,Kabol,1780000
2,Qandahar,AFG,Qandahar,237500
3,Herat,AFG,Herat,186800
4,Mazar-e-Sharif,AFG,Balkh,127800
5,Amsterdam,NLD,Noord-Holland,731200
6,Rotterdam,NLD,Zuid-Holland,593321
7,Haag,NLD,Zuid-Holland,440900
8,Utrecht,NLD,Utrecht,234323
9,Eindhoven,NLD,Noord-Brabant,201843


- 검색: 국가코드 KOR의 인구수 Top 10

In [11]:
cur =conn.cursor()
sql ="SELECT * FROM city  where countrycode=%s order by population LIMIT 10;"
cur.execute(sql,('KOR',))          # 파라메터는 반드시 튜플로 전달해야함
rows=cur.fetchall()
cur.close()


In [12]:
for row in rows:
    print(row)

(2400, 'Mun-gyong', 'KOR', 'Kyongsangbuk', 92239)
(2399, 'Tonghae', 'KOR', 'Kang-won', 95472)
(2398, 'Namwon', 'KOR', 'Chollabuk', 103544)
(2397, 'Naju', 'KOR', 'Chollanam', 107831)
(2396, 'Uiwang', 'KOR', 'Kyonggi', 108788)
(2395, 'Sachon', 'KOR', 'Kyongsangnam', 113494)
(2394, 'Yongchon', 'KOR', 'Kyongsangbuk', 113511)
(2393, 'Kimje', 'KOR', 'Chollabuk', 115427)
(2392, 'Hanam', 'KOR', 'Kyonggi', 115812)
(2391, 'Miryang', 'KOR', 'Kyongsangnam', 121501)


In [22]:
cur =conn.cursor()
sql ="SELECT * FROM city  where name=%s;"
cur.execute(sql,('수원',))          # 파라메터는 반드시 튜플로 전달해야함
row=cur.fetchone()                     # 하나의 레코드
cur.close()
row

(2383, '수원', 'KOR', 'Kyonggi', 1200000)

- 데이터 추가(Insert)

In [14]:
cur =conn.cursor()
sql ="INSERT INTO city VALUES(DEFAULT ,'자바','KOR','kyonggi',6000000);"
cur.execute(sql)  
conn.commit()           # DB가 변화되는 sql(insert, update,Delete)에서는 필수       
cur.close()

In [15]:
params = ('노드','KOR','kyonggi',6000000)

cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT,%s,%s,%s,%s);"
cur.execute(sql,params)
conn.commit() 
cur.close()

In [16]:
params_list = [
    ('HTML','KOR','kyonggi',7000000),('CSS','KOR','kyonggi',7200000)
]
cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT,%s,%s,%s,%s);"
#for params in params_list:
#   cur.execute(sql,params)
cur.executemany(sql,params_list)          # 몇건은 가능하나,100건이상은 어려움
conn.commit() 
cur.close()

- 데이터 수정(Update)

In [17]:
cur = conn.cursor()
sql = "UPDATE city SET `Name`='CSS3' WHERE id=5008;"
cur.execute(sql)
conn.commit() 
cur.close()

In [19]:
params_list =[('HTML5',5007),('HTML5',5009)]
cur = conn.cursor()
sql = "UPDATE city SET `Name`=%s WHERE id=%s;"
cur.executemany(sql,params_list)
conn.commit() 
cur.close()

- 데이터 삭제

In [20]:
cur = conn.cursor()
sql = "DELETE FROM city where id >%s;"
cur.executemany(sql,(5000,))
conn.commit() 
cur.close()

#### 4. MySql 접속 해제

In [23]:
conn.close()